<a href="https://colab.research.google.com/github/Dmitri9149/TensorFlow-PyTorch-basics/blob/master/TensorFlow_Padding_Stride_MultiChannels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:
# -U: Upgrade all packages to the newest available version
!pip install -U d2l
from d2l import tensorflow as d2l
import tensorflow as tf

Requirement already up-to-date: d2l in /usr/local/lib/python3.6/dist-packages (0.15.1)


Exercises and some code modification/experimentation for the d2l.ai book : http://d2l.ai/ 

Padding

In [133]:
### initialize convolutional layer , make some changes in sizes of the layer 
### with the help of padding
def comp_conv2d(conv2d, X):
  ### (1,1) correspond to batch size and num of channels
  X = tf.reshape(X, (1,) +  X.shape + (1,))
  Y = conv2d(X)
### elim dimentions we do not need
  return tf.reshape(Y,Y.shape[1:3])

In [134]:
conv2d=tf.keras.layers.Conv2D(1, kernel_size = 3, padding = 'same')
X = tf.random.uniform(shape=(8,8))
comp_conv2d(conv2d,X).shape

TensorShape([8, 8])

Stride

In [135]:
conv2d = tf.keras.layers.Conv2D(1, kernel_size=3, padding='same', strides=2)
comp_conv2d(conv2d, X).shape

TensorShape([4, 4])

In [136]:
conv2d = tf.keras.layers.Conv2D(1, kernel_size=(3,5), padding='valid',
                                strides=(3, 4))
comp_conv2d(conv2d, X).shape

TensorShape([2, 1])

MultiChannels

In [137]:
### input X and kernel K , compute cross correlation
def corr2d(X, K):  #@save
    h, w = K.shape
    Y = tf.Variable(tf.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1)))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j].assign(tf.reduce_sum(
                X[i: i + h, j: j + w] * K))
    return Y

In [138]:
def corr2d_multi_in(X,K):
## iterate via the first dimention (channels); sum results together
## zip 'list' of 2D inputs and kernels:
  return tf.reduce_sum([corr2d(x,k) for x,k in zip(X,K) ], axis = 0)

In [139]:
X = tf.constant([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = tf.constant([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 56.,  72.],
       [104., 120.]], dtype=float32)>

In [140]:
### multiple output and input
### size of input is (i, h, w) size of kernel is (o,i,h_k,w_k) 
def corr2d_multi_in_out(X,K):
## iterate via o -> first dim of kernel and at every step make cross correlation 
### with input X
############# iterate over first dim of K 
  return tf.stack([corr2d_multi_in(X,k) for k in K], 0)

In [141]:
K = tf.stack([K,K+1,K+2])
K.shape

TensorShape([3, 2, 2, 2])

In [142]:
corr2d_multi_in_out(X, K)

<tf.Tensor: shape=(3, 2, 2), dtype=float32, numpy=
array([[[ 56.,  72.],
        [104., 120.]],

       [[ 76., 100.],
        [148., 172.]],

       [[ 96., 128.],
        [192., 224.]]], dtype=float32)>

1x1 Convolutional Layer

In [143]:
def corr2d_multi_in_out_1x1(X,K):
  i,h,w=X.shape
  o=K.shape[0]
  X=tf.reshape(X,(i,h*w))
  K=tf.reshape(K,(o,i))
  Y=tf.matmul(K,X)
  return tf.reshape(Y,(o,h,w))



In [144]:
X = tf.random.normal((2, 3, 3), 0, 1)
K = tf.random.normal((4, 2, 1, 1), 0, 1)

In [145]:
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(tf.reduce_sum(tf.abs(Y1 - Y2))) < 1e-6

In [146]:
Y1

<tf.Tensor: shape=(4, 3, 3), dtype=float32, numpy=
array([[[-0.14064856, -0.2857571 ,  1.5360472 ],
        [-2.1105278 , -0.7735818 ,  0.39949942],
        [ 0.9469091 , -2.274635  ,  0.03773113]],

       [[ 0.18746793,  0.6327612 ,  0.1844825 ],
        [ 0.15347868, -0.3948664 , -0.24932325],
        [-0.19207533,  0.15931202,  0.16920434]],

       [[ 0.43389654,  1.3641801 , -0.46221122],
        [ 1.4148483 , -0.3458023 , -0.68987674],
        [-0.8708789 ,  1.5131727 ,  0.30417553]],

       [[-0.60983616, -1.7373915 ,  2.244094  ],
        [-3.8886058 , -0.5327018 ,  1.1719078 ],
        [ 1.9884608 , -4.1788993 , -0.2707047 ]]], dtype=float32)>